# Create master df
Total observations = 50,000
<br>0 = 25,000, 15,000 from sentiment 140, 10,000 from amazon reviews
<br>1 = 25,000, 10,000 from sentiment 140, 5000 from consumer complaints, 10000 from amazon reviews
<br> All observations from sentiment 140_neutlabels

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', 0) #To display entire text content of a column
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
from sklearn.model_selection import train_test_split
import os
from functools import reduce

In [4]:
os.chdir('D:\capstone_data\processed_csv\duplicates_rm')

In [3]:
#Read all dfs
sentiment_140_df = pd.read_csv('sentiment140_no_dplcts_textcol.csv')
sentiment_140_neutlabels =  pd.read_csv('sentiment_140_neutlabels_no_dplcts_textcol.csv')
consumer_complaints_df = pd.read_csv('consumer_complaints_no_dplcts_textcol.csv')
amazonreviews_df = pd.read_csv('amazonreviews_with_label_no_dplcts_textcol.csv')

In [5]:
#Extract 15,000 0s from sentiment 140
sentiment_140_0s_15k = sentiment_140_df[sentiment_140_df['label']==0].sample(
    n = 15000, replace = False, random_state = 7)
sentiment_140_0s_15k['label'].value_counts()

0    15000
Name: label, dtype: int64

In [6]:
#Extract 10,000 0s from amazon reviews
amazonreviews_0s_10k = amazonreviews_df[amazonreviews_df['label'] == 0].sample(
n = 10000, replace = False, random_state = 7)
amazonreviews_0s_10k['label'].value_counts()

0    10000
Name: label, dtype: int64

In [7]:
#Extract all 0s from sentiment_140_neutlabels
sentiment_140_neutlabels_0s = sentiment_140_neutlabels[sentiment_140_neutlabels['label'] == 0]
sentiment_140_neutlabels_0s['label'].value_counts()

0    321
Name: label, dtype: int64

In [8]:
#Concat all 0s in master df 
master_df = pd.concat([sentiment_140_0s_15k, amazonreviews_0s_10k, sentiment_140_neutlabels_0s],
                     axis = 0, ignore_index = True)

In [9]:
#Verify
display(master_df.tail(2))
display(master_df['label'].value_counts())

,text,label
25319,Ask Programming: LaTeX or InDesign?: submitted by calcio1 [link] [1 comment] http://tinyurl.com/myfmf7,0
25320,Ahhh... back in a *real* text editing environment. I &lt;3 LaTeX.,0


0    25321
Name: label, dtype: int64

In [10]:
#Create a function to extract data points from df
def extract_data_pts(df, label, n_pts):
    df = df[df['label'] == label].sample(
    n = n_pts, replace = False, random_state = 7)
    return df

In [11]:
#Extract 10,000 1s from sentiment 140
sentiment_140_1s_10k = extract_data_pts(sentiment_140_df,1,10000)
display(sentiment_140_1s_10k['label'].value_counts())
display(sentiment_140_1s_10k.head(2))

1    10000
Name: label, dtype: int64

,text,label
548091,MISSING MY BOYFRIEND,1
163918,@SquishT77 So sorry to hear about your mum.,1


In [12]:
#Extract 5,000 1s from consumer complaints
consumer_complaints_1s_5k = extract_data_pts(consumer_complaints_df,1,5000)
consumer_complaints_1s_5k['label'].value_counts()

1    5000
Name: label, dtype: int64

In [13]:
#Extract 10,000 1s from amazon
amazon_reviews_1s_10k = extract_data_pts(amazonreviews_df,1,10000)
amazon_reviews_1s_10k['label'].value_counts()

1    10000
Name: label, dtype: int64

In [14]:
#Extract all 1s from amazon_reviews_1s_10k
sentiment_140_neutlabels_1s = sentiment_140_neutlabels[sentiment_140_neutlabels['label'] == 1]
sentiment_140_neutlabels_1s['label'].value_counts()

1    177
Name: label, dtype: int64

In [15]:
#Concat all 4 dfs above and append to master_df
master_df = master_df.append(
    pd.concat([sentiment_140_1s_10k,consumer_complaints_1s_5k,
    amazon_reviews_1s_10k, sentiment_140_neutlabels_1s],
                          axis = 0, ignore_index = True)
    , ignore_index = True)

In [16]:
#Verify master_df
display(master_df.shape)
display(master_df.tail(2))
display(master_df['label'].value_counts())

(50498, 2)

,text,label
50496,"Trouble in Iran, I see. Hmm. Iran. Iran so far away. #flockofseagullsweregeopoliticallycorrect",1
50497,Reading the tweets coming out of Iran... The whole thing is terrifying and incredibly sad...,1


0    25321
1    25177
Name: label, dtype: int64

In [18]:
#Write master_df to csv
master_df.to_csv('master_df_snmnt_&cmplnts.csv', index = False)

# Data Cleaning 

In [6]:
#Read 
master_df = pd.read_csv('master_df_snmnt_&cmplnts.csv')
master_df.tail(100).head()

,text,label
50398,Naive Bayes using EM for Text Classification. Really Frustrating...,1
50399,"@KarrisFoxy If you're being harassed by calls about your car warranty, changing your number won't fix that. They call every number. #d-bags",1
50400,Just blocked United Blood Services using Google Voice. They call more than those Car Warranty guys.,1
50401,#at&amp;t is complete fail.,1
50402,@broskiii OH SNAP YOU WORK AT AT&amp;T DON'T YOU,1


In [39]:
s="haaaappppyy"

In [45]:
reduce(lambda x,y: x+y,s)

'haaaappppyy'

In [24]:
reduce(lambda x,y: x+y if x[-2:]!=y*2 else x, s, "")


'haappyy'

In [ ]:
reduce(lambda x,y: x+y if x[-2:]!=y*2 else x, s, "")
'haappyy'